In [1]:
!pip install kafka-python

In [2]:
import json
import time 
from kafka import KafkaProducer

### Connecting to the Kafka server

In [5]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'


topic_name = 'test-topic'

In [6]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [7]:
producer.bootstrap_connected()

True

### Sending data to the stream

In [8]:
t0 = time.time()

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
    
t1 = time.time()
print(f'sending messages took {(t1 - t0):.2f} seconds')

t0 = time.time()
producer.flush()
t1 = time.time()
print(f'flushing took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
sending messages took 0.55 seconds
flushing took 0.00 seconds


### Sending the taxi data

In [9]:
import pandas as pd

In [11]:
url = 'green_tripdata_2019-10 (1).csv/green_tripdata_2019-10.csv'
df = pd.read_csv(url)

C:\Users\DELL\AppData\Local\Temp\ipykernel_14464\713696256.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [12]:
df_green = df[['lpep_pickup_datetime',
'lpep_dropoff_datetime',
'PULocationID',
'DOLocationID',
'passenger_count',
'trip_distance',
'tip_amount']]

In [13]:
df_green.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [14]:
green_topic = "green-trips-2"

t0 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(green_topic, value=row_dict)
t1 = time.time()

producer.flush()

print(f"DONE in {t1-t0} seconds")

DONE in 77.25654220581055 seconds


### Creating the PySpark consumer

In [18]:
import findspark
findspark.init()

In [19]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [20]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips-2") \
    .option("startingOffsets", "earliest") \
    .load()

In [21]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

Row(lpep_pickup_datetime='2019-10-01 00:57:24', lpep_dropoff_datetime='2019-10-01 01:08:01', PULocationID=168, DOLocationID=60, passenger_count=1.0, trip_distance=2.43, tip_amount=0.0)


In [22]:
query.stop()

### Parsing the data

In [23]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [24]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [25]:
query_2 = green_stream.writeStream.foreachBatch(peek).start()

In [26]:
from pyspark.sql.functions import current_timestamp, window, col, count

In [27]:
green_stream = green_stream.withColumn("timestamp", current_timestamp())

### Most popular destination

In [28]:
from pyspark.sql.functions import window, col, count

popular_destinations = (green_stream
          .groupBy(window(col("timestamp"), "5 minutes"), "DOLocationID")
          .agg(count("*").alias("count"))
          .orderBy(col("count").desc())
         )

In [29]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()